In [138]:
import numpy as np
import openmc

# Question 1

In [124]:
# data
mf = 32_000  # mass feed, kgU/day
mt = 26_000  # mass tails, kg tails/day

xf = 0.7 / 100  # feed enrichment
xt = 0.25 / 100  # tails enrichment

# calculations
uf = mf * xf  # u mass feed 
ut = mt * xt  # u mass tails

mp = mf - mt  # mass enriched
up = uf - ut  # u enriched

xp = up / mp  # product enrichment

print(f"Product Assay: {round(xp, 4) * 100}%")
assert(mp * xp + mt * xt == mf * xf)

Product Assay: 2.65%


# Question 2

In [46]:
# data
xf = 0.7 / 100
xp = 2.9 / 100

mf = 35000
mp = 6000

# calculations
# find tail mass and tail enrichment
uf = mf * xf
up = mp * xp

mt = mf - mp
ut = uf - up
xt = ut / mt

print(f"Tails Mass: {mt} kg")
print(f"Tails Assay: {round(xt, 4) * 100}%")

assert(ut + up == uf)
assert(mt + mp == mf)
assert(mp * xp + mt * xt == mf * xf)

Tails Mass: 29000 kg
Tails Assay: 0.24%


# Question 3

In [153]:
# data
m_b10 = openmc.data.atomic_mass("B10")
m_b11 = openmc.data.atomic_mass("B11")

[(_, xi_b10), (_, xi_b11)] = openmc.data.isotopes("B")

def mass_percent(xi1, u1, u2):
    m1 = u1 * xi1
    xi2 = 1 - xi1
    m2 = u2 * xi2

    xm1 = m1 / (m1 + m2)
    xm2 = m2 / (m1 + m2)
    return xm1

# calculations
xf = 20 / 100 # assuming 20% mass enrichment
xp = 90 / 100
xt = 8 / 100

xf = mass_percent(xf, m_b10, m_b11)
xp = mass_percent(xp, m_b10, m_b11)
xt = mass_percent(xt, m_b10, m_b11)

mp = 1

# calculations
mf = (xp - xt) / (xf - xt) * mp
print(f"Feed Mass: {round(mf, 2)} kg")

Feed Mass: 7.3 kg


## part b
- B10 absorbs a neutron and goes to Li7
- Li7 decays into alpha + T

## part c
- 

# Question 4

In [133]:
# data
xh = 90 / 100   # acts like product
xd = 0.2 / 100  # acts like waste
xf = 4 / 100    # acts like feed

# calculations
mf = ((xh - xd) / (xf - xd))

print(f"Mass Fuel: {round(mf, 2)} kg")

Mass Fuel: 23.63 kg


# Question 5

In [73]:
# data
x235 = 0.711 / 100
x238 = 1 - x_u235

# from Q6, ignoring U234, U236
th235 = 7.1e8 # years
th238 = 4.51e9 # years

# decay constants
lam235 = np.log(2) / th235
lam238 = np.log(2) / th238

# past mass
time = 1.8e9
pm235 = x235 * np.exp(lam235 * time)
pm238 = x238 * np.exp(lam238 * time)

# past enrichment
past_x235 = pm235 / (pm235 + pm238) * 100

print(f"Past U235 Enrichment: {round(past_x235, 2)}%")

Past U235 Enrichment: 3.05%


# Question 6

In [137]:
# current enrichments
x235 = 0.711 / 100
x238 = 1 - x_u235

# ignoring U234, U236
th235 = 7.1e8 # years
th238 = 4.51e9 # years

# decay constants
lam235 = np.log(2) / th235
lam238 = np.log(2) / th238

# future mass
time = 1e9
fm235 = x235 * np.exp(-lam235 * time)
fm238 = x238 * np.exp(-lam238 * time)

# future enrichments
future_x235 = fm235 / (fm235 + fm238)
future_x238 = fm238 / (fm235 + fm238)

# seperation potential, swu
def sep_pot(x):
    return (2 * x - 1) * np.log(x / (1 - x))

# swu
def swu(xf, xp, xw):
    p = 1
    f = (xp - xw) / (xf - xw)
    w = (xp - xf) / (xf - xw)

    vp = sep_pot(xp)
    vf = sep_pot(xf)
    vw = sep_pot(xw)

    return 1 * (p * vp + w * vw - f * vf)

future_swu = swu(future_x235, xp=4.5/100, xw=0.2/100)
print(f"Future SWU: {round(future_swu, 2)} SWU")

Future SWU: 14.09 SWU


# Question 7

find price of U3O8 where it will cost less to enrich 0.2% U235 tails than mine new uranium

In [103]:
# prices
p_swu = 80  # $/SWU
p_con = 4  # $/kgU

# enrichments
xf = 0.711 / 100
xp = 3 / 100
xw = 0.1 / 100

xt = 0.2 / 100

# cost of enriching tails
swu_tails = swu(xt, xp, xw)
c_tails = p_swu * swu_tails

# mass fraction of U / U3O8
m_u = 238.02891
m_o = 15.999

mass_frac = (3 * m_u) / (3 * m_u + 8 * m_o)

# cost of converting and enriching new U3O8
swu_new = swu(xf, xp, xw)

#c_new = p_swu * swu_ce + p_u3o8 / mass_frac
p_u3o8 = (c_tails - p_swu * swu_ce) * mass_frac  # $/kg
p_u3o8 /= 2.2  # kg/lbs

print(f"Price Yellowcake: ${round(p_u3o8, 2)}/lb")

Price Yellowcake: $334.38/lb


# Question 8

In [114]:
# part a
m_tails = 7e8

# enrichments
x_tails = 0.25 / 100
x_fuel = 4.5 / 100
x_waste = 0.15 / 100

# mass of product
# tails are feed, fuel is product, waste is waste
mp = m_tails / ((x_fuel - x_waste) / (x_tails - x_waste))
print(f"Mass of Product: {round(mp * 1e-6, 3)}M kg")

Mass of Product: 16.092M kg


In [154]:
# part b
p_swu = 80
swu_tails = swu(x_tails, x_fuel, x_waste)

p_fuel = swu_tails * p_swu
print(f"Cost per kg: ${round(p_fuel, 2)}/kg")

Cost per kg: $1521.4/kg
